# Standard Keras Optimizers w/ Cyclic Learning Rate

In [21]:
from clr_optimizers import *

import numpy as np

## Toy CLR Function

In [19]:
def lr(iterations):
    """
    Test CLR function w/ following params:
    """
    step_size = 2000.
    lr = 0.01
    max_lr = 0.06
    cycle = np.floor(1+iterations/(2*step_size))
    x = np.abs(iterations/step_size - 2*cycle + 1)
    return lr + (max_lr-lr)*np.maximum(0, (1-x))

In [20]:
for iteration in range(1000, 11000, 1000):
    print(f"Learning rate at iteration {iteration}: {lr(iteration)}")

Learning rate at iteration 1000: 0.034999999999999996
Learning rate at iteration 2000: 0.06
Learning rate at iteration 3000: 0.034999999999999996
Learning rate at iteration 4000: 0.01
Learning rate at iteration 5000: 0.034999999999999996
Learning rate at iteration 6000: 0.06
Learning rate at iteration 7000: 0.034999999999999996
Learning rate at iteration 8000: 0.01
Learning rate at iteration 9000: 0.034999999999999996
Learning rate at iteration 10000: 0.06


## Test Model

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input

In [3]:
inp = Input(shape=(15,))                
x = Dense(10, activation='relu')(inp)
x = Dense(1, activation='sigmoid')(x)

model = Model(inp, x)

In [4]:
clr = {
    'max_lr': 0.06,
    'step_size': 2000
}

In [6]:
X = np.random.rand(2000000,15)

In [7]:
Y = np.random.randint(0,2,size=2000000)

## CLR Tests

Set batch size so number of iterations in one epoch is 1000, half the default step_size (2000).

The Current LR value should be ~ 0.01, 0.06, and 0.035 for even multiples of 2000, odd multiples of 2000, and odd multiples of 1000 (1/2 step_size) respectively.

Checked for each model. Note that "current_lr" is updated immediately following cycle update, and should follow expectations. It would only be different if rate-decay is in effect, as this happens prior to cycle.

### SGD

In [12]:
model.compile(optimizer=SGD(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### RMSProp

In [13]:
model.compile(optimizer=RMSprop(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### Adagrad

In [14]:
model.compile(optimizer=Adagrad(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### Adadelta

In [15]:
model.compile(optimizer=Adadelta(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### Adam

In [16]:
model.compile(optimizer=Adam(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### Adamax

In [17]:
model.compile(optimizer=Adamax(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


### Nadam

In [18]:
model.compile(optimizer=Nadam(lr=0.01,clr=clr), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


Values check out